In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
assert tf.__version__.startswith('2')
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
IMAGE_PATH = r"C:\Users\offco\Documents\Dev_Projects\BookCoverClassifier\Datasets\Augmented_Cover_Images"
IMAGE_SIZE = (256, 256)
BATCH_SIZE = 32
NUM_CLASSES = 

train = tf.keras.preprocessing.image_dataset_from_directory(
    IMAGE_PATH,
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)
train = train.map(lambda x, y: (x, tf.one_hot(y, depth=NUM_CLASSES)))

validation = tf.keras.preprocessing.image_dataset_from_directory(
    IMAGE_PATH,
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)
validation = validation.map(lambda x, y: (x, tf.one_hot(y, depth=NUM_CLASSES)))

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(str(labels[i].numpy()))
        plt.axis("off")

In [ ]:
train = train.prefetch(buffer_size=32)
validation = validation.prefetch(buffer_size=32)

In [ ]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)

model = make_model(input_shape=IMAGE_SIZE + (3,), num_classes=5)

In [ ]:
model.summary()

In [ ]:
epochs = 150

callbacks = [
    keras.callbacks.ModelCheckpoint(r"C:\Users\offco\Documents\Dev_Projects\BookCoverClassifier\Test_Run\Log\save_at_{epoch}.h5"),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train, epochs=epochs, callbacks=callbacks, validation_data=validation,
)

In [ ]:
SAVE_PATH = r"C:\Users\offco\Documents\Dev_Projects\BookCoverClassifier\Keras_Model"
model.save(SAVE_PATH)

In [ ]:
QMODEL_SAVE_PATH = r"C:\Users\offco\Documents\Dev_Projects\BookCoverClassifier\Quantized Model\quantized_model.tflite"

def representative_dataset():
    for data, label in train.take(30):
        yield [np.array(data, dtype=np.float32)]

converter = tf.lite.TFLiteConverter.from_saved_model(SAVE_PATH)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_quant_model = converter.convert()

with open(QMODEL_SAVE_PATH, 'wb') as f:
    f.write(tflite_quant_model)